In [1]:
!nvidia-smi

Mon May 31 06:05:59 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.51.06    Driver Version: 450.51.06    CUDA Version: 11.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P0    28W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
import sys
try:
    import resnest
except ModuleNotFoundError:
    !pip install -q "../input/resnest50-fast-package/resnest-0.0.6b20200701/resnest"
sys.path.append('../input/timm-pytorch-image-models/pytorch-image-models-master')
sys.path.append('../input/efficientnet-pytorch/EfficientNet-PyTorch/EfficientNet-PyTorch-master')

In [3]:
import numpy as np
import librosa as lb
import soundfile as sf
import pandas as pd
import cv2
from pathlib import Path
import re
import random
import torch
from torch import nn
from torchvision import transforms
from  torch.utils.data import Dataset, DataLoader
import timm
from tqdm.notebook import tqdm
import time
from resnest.torch import resnest50
from efficientnet_pytorch import EfficientNet
from itertools import combinations, permutations
import torchvision

# Configs

In [4]:
NUM_CLASSES = 397
SR = 32_000
DURATION = 5
COMBINATIONS = 7
THRESHs = [ 0.2, 0.3, 0.4, 0.5, 0.55, 0.6, 0.65 ]



DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("DEVICE:", DEVICE)

TEST_AUDIO_ROOT = Path("../input/birdclef-2021/test_soundscapes")
SAMPLE_SUB_PATH = "../input/birdclef-2021/sample_submission.csv"
TARGET_PATH = None
    
if not len(list(TEST_AUDIO_ROOT.glob("*.ogg"))):
    TEST_AUDIO_ROOT = Path("../input/birdclef-2021/train_soundscapes")
    SAMPLE_SUB_PATH = None
    TARGET_PATH = Path("../input/birdclef-2021/train_soundscape_labels.csv")

DEVICE: cuda


# Utils

In [5]:
def get_metrics(s_true, s_pred):
    s_true = set(s_true.split())
    s_pred = set(s_pred.split())
    n, n_true, n_pred = len(s_true.intersection(s_pred)), len(s_true), len(s_pred)
    
    prec = n/n_pred
    rec = n/n_true
    f1 = 2*prec*rec/(prec + rec) if prec + rec else 0
    
    return {"f1": f1, "prec": prec, "rec": rec, "n_true": n_true, "n_pred": n_pred, "n": n}

@torch.no_grad()
def get_thresh_preds(out, thresh=None):
    thresh = thresh or THRESH
    o = (-out).argsort(1)
    npreds = (out > thresh).sum(1)
    preds = []
    for oo, npred in zip(o, npreds):
        preds.append(oo[:npred].cpu().numpy().tolist())
    return preds

def get_bird_names(preds):
    bird_names = []
    for pred in preds:
        if not pred:
            bird_names.append("nocall")
        else:
            bird_names.append(" ".join([INV_LABEL_IDS[bird_id] for bird_id in pred]))
    return bird_names

def predict(nets, test_data, names=True, threshold=0.5):
    preds = []
    with torch.no_grad():
        for idx in  tqdm(list(range(len(test_data)))):
            xb = torch.from_numpy(test_data[idx]).to(DEVICE)
            pred = 0.
            for net in nets:
                o = net(xb)
                o = torch.sigmoid(o)

                pred += o

            pred /= len(nets)
            
            if names:
                pred = get_bird_names(get_thresh_preds(pred,threshold))

            preds.append(pred)
    return preds

def preds_as_df(data, preds):
    sub = {
        "row_id": [],
        "birds": [],
    }
    
    for row, pred in zip(data.itertuples(False), preds):
        row_id = [f"{row.id}_{row.site}_{5*i}" for i in range(1, len(pred)+1)]
        sub["birds"] += pred
        sub["row_id"] += row_id
        
    sub = pd.DataFrame(sub)
    
    if SAMPLE_SUB_PATH:
        sample_sub = pd.read_csv(SAMPLE_SUB_PATH, usecols=["row_id"])
        sub = sample_sub.merge(sub, on="row_id", how="left")
        sub["birds"] = sub["birds"].fillna("nocall")
    return sub

# Data

In [6]:
class MelSpecComputer:
    def __init__(self, sr, n_mels, fmin, fmax, **kwargs):
        self.sr = sr
        self.n_mels = n_mels
        self.fmin = fmin
        self.fmax = fmax
        kwargs["n_fft"] = kwargs.get("n_fft", self.sr//10)
        kwargs["hop_length"] = kwargs.get("hop_length", self.sr//(10*4))
        self.kwargs = kwargs

    def __call__(self, y):

        melspec = lb.feature.melspectrogram(
            y, sr=self.sr, n_mels=self.n_mels, fmin=self.fmin, fmax=self.fmax, **self.kwargs,
        )

        melspec = lb.power_to_db(melspec).astype(np.float32)
        return melspec

In [7]:
def mono_to_color(X, eps=1e-6, mean=None, std=None):
    mean = mean or X.mean()
    std = std or X.std()
    X = (X - mean) / (std + eps)
    
    _min, _max = X.min(), X.max()

    if (_max - _min) > eps:
        V = np.clip(X, _min, _max)
        V = 255 * (V - _min) / (_max - _min)
        V = V.astype(np.uint8)
    else:
        V = np.zeros_like(X, dtype=np.uint8)
    
    return V

def random_power(images, power = 1.5, c= 0.7):
    images = images - images.min()
    images = images/(images.max()+0.0000001)
    images = images**(random.random()*power + c)
    return images

def crop_or_pad(y, length):
    if len(y) < length:
        y = np.concatenate([y, length - np.zeros(len(y))])
    elif len(y) > length:
        y = y[:length]
    return y

In [8]:
class BirdCLEFDataset(Dataset):
    def __init__(self, data, sr=SR, n_mels=128, fmin=0, fmax=None, duration=DURATION, step=None, res_type="kaiser_fast", resample=True):
        
        self.data = data
        
        self.sr = sr
        self.n_mels = n_mels
        self.fmin = fmin
        self.fmax = fmax or self.sr//2

        self.duration = duration
        self.audio_length = self.duration*self.sr
        self.step = step or self.audio_length
        
        self.res_type = res_type
        self.resample = resample

        self.mel_spec_computer = MelSpecComputer(sr=self.sr, n_mels=self.n_mels, fmin=self.fmin,
                                                 fmax=self.fmax)
    def __len__(self):
        return len(self.data)
    
    @staticmethod
    def normalize(image):
        image = image.astype("float32", copy=False) / 255.0
        image = np.stack([image, image, image])
        return image
    
    def audio_to_image(self, audio):
        melspec = self.mel_spec_computer(audio)
        image = random_power(melspec)
        image = mono_to_color(image)
        image = self.normalize(image)
        return image

    def read_file(self, filepath):
        audio, orig_sr = sf.read(filepath, dtype="float32")

        if self.resample and orig_sr != self.sr:
            audio = lb.resample(audio, orig_sr, self.sr, res_type=self.res_type)
          
        audios = []
        for i in range(self.audio_length, len(audio) + self.step, self.step):
            start = max(0, i - self.audio_length)
            end = start + self.audio_length
            audios.append(audio[start:end])
            
        if len(audios[-1]) < self.audio_length:
            audios = audios[:-1]
            
        images = [self.audio_to_image(audio) for audio in audios]
        images = np.stack(images)

        return images
    
        
    def __getitem__(self, idx):
        return self.read_file(self.data.loc[idx, "filepath"])

In [9]:
data = pd.DataFrame(
     [(path.stem, *path.stem.split("_"), path) for path in Path(TEST_AUDIO_ROOT).glob("*.ogg")],
    columns = ["filename", "id", "site", "date", "filepath"]
)
print(data.shape)
data.head()

(20, 5)


,filename,id,site,date,filepath
0,20152_SSW_20170805,20152,SSW,20170805,../input/birdclef-2021/train_soundscapes/20152...
1,57610_COR_20190904,57610,COR,20190904,../input/birdclef-2021/train_soundscapes/57610...
2,7843_SSW_20170325,7843,SSW,20170325,../input/birdclef-2021/train_soundscapes/7843_...
3,42907_SSW_20170708,42907,SSW,20170708,../input/birdclef-2021/train_soundscapes/42907...
4,7019_COR_20190904,7019,COR,20190904,../input/birdclef-2021/train_soundscapes/7019_...


In [10]:
df_train = pd.read_csv("../input/birdclef-2021/train_metadata.csv")

LABEL_IDS = {label: label_id for label_id,label in enumerate(sorted(df_train["primary_label"].unique()))}
INV_LABEL_IDS = {val: key for key,val in LABEL_IDS.items()}

# Inference

In [11]:
test_data = BirdCLEFDataset(data=data)
len(test_data), test_data[0].shape

(20, (120, 3, 128, 201))

In [12]:
def get_model(name, num_classes=NUM_CLASSES):
    """
    Loads a pretrained model. 
    Supports ResNest, ResNext-wsl, EfficientNet, ResNext and ResNet.

    Arguments:
        name {str} -- Name of the model to load

    Keyword Arguments:
        num_classes {int} -- Number of classes to use (default: {1})

    Returns:
        torch model -- Pretrained model
    """
    if "resnest" in name:
        model = getattr(resnest_torch, name)(pretrained=True)
    elif "wsl" in name:
        model = torch.hub.load("facebookresearch/WSL-Images", name)
    elif name.startswith("resnext") or  name.startswith("resnet"):
        model = torch.hub.load("pytorch/vision:v0.6.0", name, pretrained=True)
    elif name.startswith("tf_efficientnet_b"):
        model = getattr(timm.models.efficientnet, name)(pretrained=False)
    elif "efficientnet-b" in name:
        model = EfficientNet.from_pretrained(name)
    else:
        model = pretrainedmodels.__dict__[name](pretrained='imagenet')

    if hasattr(model, "fc"):
        nb_ft = model.fc.in_features
        model.fc = nn.Linear(nb_ft, num_classes)
    elif hasattr(model, "_fc"):
        nb_ft = model._fc.in_features
        model._fc = nn.Linear(nb_ft, num_classes)
    elif hasattr(model, "classifier"):
        nb_ft = model.classifier.in_features
        model.classifier = nn.Linear(nb_ft, num_classes)
    elif hasattr(model, "last_linear"):
        nb_ft = model.last_linear.in_features
        model.last_linear = nn.Linear(nb_ft, num_classes)

    return model

In [13]:
def load_net(checkpoint_path, num_classes=NUM_CLASSES):
    if "efficientnet-b0" in checkpoint_path:
        net = EfficientNet.from_name('efficientnet-b0', num_classes=num_classes).cuda()
        #print(net)
        if hasattr(net, "fc"):
            nb_ft = net.fc.in_features
            net.fc = nn.Linear(nb_ft, num_classes)
        elif hasattr(net, "_fc"):
            nb_ft = net._fc.in_features
            net._fc = nn.Linear(nb_ft, num_classes)
        elif hasattr(net, "classifier"):
            nb_ft = net.classifier.in_features
            net.classifier = nn.Linear(nb_ft, num_classes)
        elif hasattr(net, "last_linear"):
            nb_ft = net.last_linear.in_features
            net.last_linear = nn.Linear(nb_ft, num_classes)
    if "resnext" in checkpoint_path:
        net = torchvision.models.resnext50_32x4d(pretrained=False)
        net.fc = nn.Linear(net.fc.in_features, num_classes)
    if "tf_efficientnet_b4" in checkpoint_path:
        net = timm.create_model("tf_efficientnet_b4", pretrained=False)
        net.classifier = nn.Linear(net.classifier.in_features, num_classes)
    if "resnest50" in checkpoint_path:
        net = resnest50(pretrained=False)
        net.fc = nn.Linear(net.fc.in_features, num_classes)
    dummy_device = torch.device("cpu")
    d = torch.load(checkpoint_path, map_location=dummy_device)
    for key in list(d.keys()):
        d[key.replace("model.", "")] = d.pop(key)
    net.load_state_dict(d)
    net = net.to(DEVICE)
    net = net.eval()
    return net

In [14]:

checkpoint_paths = [
    Path("../input/kkiller-birdclef-models-public/birdclef_resnest50_fold0_epoch_10_f1_val_06471_20210417161101.pth"),
    Path('../input/pytorch-models-birdclef/birdclef_efficientnet-b0_fold0_epoch_17_f1_val_05482_with_augs.pth'),
    Path('../input/pytorch-models-birdclef/birdclef_efficientnet-b0_fold1_epoch_18_f1_val_05420_20210525140424.pth'),
    Path('../input/pytorch-models-birdclef/birdclef_efficientnet-b0_fold2_epoch_17_f1_val_05497_20210525142055.pth'),
    Path('../input/pytorch-models-birdclef/birdclef_resnest50_fold0_epoch_17_f1_val_06073_with_augs.pth'),
    Path('../input/pytorch-models-birdclef/birdclef_efficientnet-b0_fold3_epoch_21_f1_val_05505_20210526062447.pth'),
    Path('../input/pytorch-models-birdclef/birdclef_efficientnet-b0_fold4_epoch_27_f1_val_05724_20210526065158.pth'),
    Path('../input/pytorch-models-birdclef/birdclef_resnest50_fold1_epoch_24_f1_val_06008_20210526112320.pth'),
    Path('../input/pytorch-models-birdclef/birdclef_resnext_fold0_epoch_22_f1_val_04837_20210530220016.pth')
]


# Try postprocessing

In [15]:
F1 = None
Precision = None
best_thres = None
best_combinations = []

test_paths = combinations(checkpoint_paths, COMBINATIONS)
#len(list(test_paths))
for paths in tqdm(test_paths):
    for THRESH in THRESHs:
        print("Predict Thresh :",THRESH)
        
        nets = [
            load_net(checkpoint_path.as_posix()) for checkpoint_path in paths
        ]
        pred_probas = predict(nets, test_data, names=False,threshold=THRESH)
        print(len(pred_probas))
        preds = [get_bird_names(get_thresh_preds(pred, thresh=THRESH)) for pred in pred_probas]

        sub = preds_as_df(data, preds)


        # Small validation
        if TARGET_PATH:
            sub_target = pd.read_csv(TARGET_PATH)
            sub_target = sub_target.merge(sub, how="left", on="row_id")

            print(sub_target["birds_x"].notnull().sum(), sub_target["birds_x"].notnull().sum())
            assert sub_target["birds_x"].notnull().all()
            assert sub_target["birds_y"].notnull().all()

            df_metrics = pd.DataFrame([get_metrics(s_true, s_pred) for s_true, s_pred in zip(sub_target.birds_x, sub_target.birds_y)])
            if F1 == None or Precision == None:
                F1 = df_metrics.f1.mean()
                Precision = df_metrics.prec.mean()

            if df_metrics.f1.mean() > F1:
                best_thres = THRESH
                print("F1-score:", df_metrics.f1.mean())
                best_combinations = paths
                print("Best combinations", best_combinations)

            if df_metrics.prec.mean() > Precision:
                best_thres = THRESH
                print("Precision-score:", df_metrics.prec.mean())
                best_combinations = paths
    print('='* 100)


0it [00:00, ?it/s]

Predict Thresh : 0.2


  0%|          | 0/20 [00:00<?, ?it/s]

20
2400 2400
Predict Thresh : 0.3


  0%|          | 0/20 [00:00<?, ?it/s]

...

20
2400 2400
Predict Thresh : 0.6


  0%|          | 0/20 [00:00<?, ?it/s]

20
2400 2400
Predict Thresh : 0.65


  0%|          | 0/20 [00:00<?, ?it/s]

20
2400 2400


In [16]:
print("Best Predict Thresh : ", best_thres)
print("Best F-1 score : ", F1)
print("Best Presicion score : ", Precision)
print("Best Combinations", best_combinations)
nets = [load_net(checkpoint_path.as_posix()) for checkpoint_path in best_combinations]
pred_probas = predict(nets, test_data, names=False,threshold=best_thres)
#print(len(pred_probas))
preds = [get_bird_names(get_thresh_preds(pred, thresh=best_thres)) for pred in pred_probas]

sub = preds_as_df(data, preds)
sub.to_csv(f"submission.csv", index=False)

Best Predict Thresh :  0.3
Best F-1 score :  0.6943611111111123
Best Presicion score :  0.7070833333333332
Best Combinations (PosixPath('../input/pytorch-models-birdclef/birdclef_efficientnet-b0_fold1_epoch_18_f1_val_05420_20210525140424.pth'), PosixPath('../input/pytorch-models-birdclef/birdclef_efficientnet-b0_fold2_epoch_17_f1_val_05497_20210525142055.pth'), PosixPath('../input/pytorch-models-birdclef/birdclef_resnest50_fold0_epoch_17_f1_val_06073_with_augs.pth'), PosixPath('../input/pytorch-models-birdclef/birdclef_efficientnet-b0_fold3_epoch_21_f1_val_05505_20210526062447.pth'), PosixPath('../input/pytorch-models-birdclef/birdclef_efficientnet-b0_fold4_epoch_27_f1_val_05724_20210526065158.pth'), PosixPath('../input/pytorch-models-birdclef/birdclef_resnest50_fold1_epoch_24_f1_val_06008_20210526112320.pth'), PosixPath('../input/pytorch-models-birdclef/birdclef_resnext_fold0_epoch_22_f1_val_04837_20210530220016.pth'))


  0%|          | 0/20 [00:00<?, ?it/s]

In [17]:
sub_target[sub_target.birds_y != "nocall"]

,row_id,site,audio_id,seconds,birds_x,birds_y
494,21767_COR_75,COR,21767,75,grekis,grekis
845,44957_COR_30,COR,44957,30,bobfly1,bobfly1
847,44957_COR_40,COR,44957,40,bobfly1,bobfly1
851,44957_COR_60,COR,44957,60,bobfly1,bobfly1
855,44957_COR_80,COR,44957,80,bobfly1,bobfly1
...,...,...,...,...,...,...
2267,51010_SSW_540,SSW,51010,540,bkcchi rebwoo,bkcchi
2270,51010_SSW_555,SSW,51010,555,bkcchi,bkcchi
2273,51010_SSW_570,SSW,51010,570,bkcchi norcar,bkcchi
2278,51010_SSW_595,SSW,51010,595,bkcchi,bkcchi


In [18]:
sub_target[sub_target.birds_x != "nocall"]

,row_id,site,audio_id,seconds,birds_x,birds_y
240,11254_COR_5,COR,11254,5,rubwre1,nocall
242,11254_COR_15,COR,11254,15,rubwre1,nocall
244,11254_COR_25,COR,11254,25,rubwre1,nocall
267,11254_COR_140,COR,11254,140,obnthr1,nocall
268,11254_COR_145,COR,11254,145,obnthr1,nocall
...,...,...,...,...,...,...
2391,54955_SSW_560,SSW,54955,560,grycat,nocall
2393,54955_SSW_570,SSW,54955,570,grycat,nocall
2394,54955_SSW_575,SSW,54955,575,chswar,nocall
2396,54955_SSW_585,SSW,54955,585,grycat,nocall
